In [ ]:
import numpy as np
from plotly import graph_objs as go

import tanh_antiderivative
import dsplib

In [ ]:
fs_hz = 48e3
signal_duration_sec = 0.2

## The tanh(x) function

In [ ]:
x = np.linspace(-3, 3)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.tanh(x), name='tanh'))
fig.update_layout(xaxis_title='Input magnitude', yaxis_title='Output magnitude')
fig.show()

## Sine wave

### Frequency impact

In [ ]:
mag = 2
t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz)

fig = go.Figure()
freq_points_hz = np.arange(300, fs_hz/2, 200)
for tone_freq_hz in freq_points_hz:
    sig = mag * np.sin(2*np.pi * tone_freq_hz * t)
    f, S_in = dsplib.calc_spectrum(sig, fs=fs_hz)
    _, S_out = dsplib.calc_spectrum(np.tanh(sig), fs=fs_hz)

    fig.add_trace(go.Scatter(x=f, y=S_in, visible=False, line=dict(color='blue'), name="Before"))
    fig.add_trace(go.Scatter(x=f, y=S_out, visible=False, line=dict(color='red', width=2, dash='dash'), name="After"))

active_ind = 1
lines_num = 2
fig.data[lines_num*active_ind].visible = True
fig.data[lines_num*active_ind+1].visible = True

steps = []
for i in range(len(freq_points_hz)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},],
        label=str(int(freq_points_hz[i]))
    )
    step["args"][0]["visible"][lines_num*i] = True
    step["args"][0]["visible"][lines_num*i+1] = True
    steps.append(step)

sliders = [dict(
    active=active_ind,
    currentvalue={"prefix": "Frequency: ", "suffix": " Hz"},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title=f"Signal spectrum before and after tanh",
    xaxis_title='Frequency, Hz',
    yaxis_title='Spectral density'
)
fig.update_layout(hovermode="x unified")
fig.show()

We can see that as the tone frequency increases, the generated by the `tanh` harmonics end up behind the `fs/2` frequency which leads to aliasing.

### Magnitude impact

In [ ]:
tone_freq_hz = 900
t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz, history_smp_num=0)
sig = np.sin(2*np.pi * tone_freq_hz * t)
f, S_in_0 = dsplib.calc_spectrum(sig, fs=fs_hz)

fig = go.Figure()
mag_points = np.arange(0.1, 5, 0.1)
for mag in mag_points:
    S_in = S_in_0 + 20*np.log10(mag)
    _, S_out = dsplib.calc_spectrum(np.tanh(mag*sig), fs=fs_hz)

    fig.add_trace(go.Scatter(x=f, y=S_in, visible=False, line=dict(color='blue'), name="Before"))
    fig.add_trace(go.Scatter(x=f, y=S_out, visible=False, line=dict(color='red', width=2, dash='dash'), name="After"))

active_ind = 1
fig.data[2*active_ind].visible = True
fig.data[2*active_ind+1].visible = True

steps = []
for i in range(len(mag_points)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],
        label=f"{mag_points[i]:.1f}"
    )
    step["args"][0]["visible"][2*i] = True
    step["args"][0]["visible"][2*i+1] = True
    steps.append(step)

sliders = [dict(
    active=active_ind,
    currentvalue={"prefix": "Magnitude: ", "suffix": ""},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title=f"Signal spectrum before and after tanh",
    xaxis_title='Frequency, Hz',
    yaxis_title='Spectral density'
)

fig.show()

### Trying to avoid the aliasing

In [ ]:
tone_freq_hz = 9100
mag = 0.5
history_samples = 3

t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz, history_smp_num=history_samples)
sig = mag * np.sin(2*np.pi * tone_freq_hz * t)
n = 5e-4*np.random.randn(len(t))  # noise
sig += n

sig_len = len(sig) - history_samples

# S is a matrix containing the delayed versions of 'sig': S[k,:] is 'sig' delayed by k taps
S = np.lib.stride_tricks.sliding_window_view(sig, sig_len)
S = np.flipud(S)

In [ ]:
ad_1_sig = tanh_antiderivative.order1(S)
ad_2_sig = tanh_antiderivative.order2(S)
ad_3_sig = tanh_antiderivative.order3(S)

f, S_in = dsplib.calc_spectrum(S[0,:], fs=fs_hz)
_, S_tanh = dsplib.calc_spectrum(np.tanh(S[0,:]), fs=fs_hz)
_, S_ad_1 = dsplib.calc_spectrum(ad_1_sig, fs=fs_hz)
_, S_ad_2 = dsplib.calc_spectrum(ad_2_sig, fs=fs_hz)
_, S_ad_3 = dsplib.calc_spectrum(ad_3_sig, fs=fs_hz)


fig = go.Figure()
fig.add_trace(go.Scatter(x=f, y=S_in, name="Before"))
fig.add_trace(go.Scatter(x=f, y=S_tanh, line=dict(width=2, dash='dash'), name="tanh"))
fig.add_trace(go.Scatter(x=f, y=S_ad_1, line=dict(width=2, dash='dot'), name="ad_1"))
fig.add_trace(go.Scatter(x=f, y=S_ad_2, line=dict(width=1.5, dash='dot'), name="ad_2"))
fig.add_trace(go.Scatter(x=f, y=S_ad_3, line=dict(width=1, dash='dot'), name="ad_3"))
fig.update_layout(hovermode="x unified")
fig.show()